In [240]:
import os
import re
import json
import time

import tiktoken
import pandas as pd
import numpy as np
from tqdm import tqdm
from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
# from langchain.chat_models import ChatOpenAI
# from langchain.memory import ConversationBufferMemory
# from langchain.chains import ConversationalRetrievalChain
# from langchain.llms import HuggingFaceHub

from textutils import extract_ref_from_text, get_number2drawing_dict, convert_ref_to_drawing_num

pd.set_option("display.max_columns", 999)
os.chdir('/Users/hayley/Documents/p4ds/patent_search')


In [241]:
import importlib
import textutils
importlib.reload(textutils)
from textutils import extract_ref_from_text, get_number2drawing_dict, convert_refs_to_drawing_num

In [165]:
# main.py 돌려서 나온 결과 다시 저장
# sample_data = pd.read_csv('data_preprocess/data.csv')
# sample_data.to_excel("data_preprocess/data.xlsx")

# 근데 코드에서 코너 케이스 때문에 처리가 안된 pdf 가 있어서 그거는 excel 파일 다운받아서 눈으로 보고 채움 -> 그러다보니까 \n이 안지워진 경우가 있어서 지웠음
# sample_data = pd.read_excel('data_preprocess/data.xlsx')
# for col in sample_data.columns:
#     sample_data[col] = sample_data[col].str.replace('\n', '')
# sample_data.to_excel("data_preprocess/data.xlsx")

In [179]:
# # negative sample 3개 추가하느라고 다시 main.py 돌리고 안읽어와진 ~605번 특허 처리
# sample_data = pd.read_csv('data_preprocess/data.csv')

# sample_data_orig = pd.read_excel("data_preprocess/sample_data.xlsx", index_col=0)
# sample_data_orig.to_excel("data_preprocess/sample_data_old.xlsx") # save old version as _old

# sample_data.loc[len(sample_data)-1, :] = sample_data_orig.loc[3, :]
# sample_data['id'] = sample_data['id'].astype(float).astype(int).astype(str)
# # add labels column
# sample_data['labels'] = ""
# sample_data.loc[sample_data['id']=='1020180014052', 'labels'] = 'source'
# sample_data.loc[sample_data['id'].isin(['1020050097605','1020177009557', '1020120156759']), 'labels'] = 'target'
# sample_data.loc[sample_data['id'].isin(['1020127022798','1020157015204', '1020200149050']), 'labels'] = 'negative'
# sample_data.to_excel("data_preprocess/sample_data.xlsx")

In [243]:
## negative sample data 를 더 추가해볼까?
sample_data = pd.read_excel("data_preprocess/sample_data_old.xlsx", index_col=0)
negative_samples = pd.read_csv('pdf_process/data.csv')
negative_samples['id'] = negative_samples['id'].astype(str)

# for i, row in negative_samples.iterrows():
#     try:
#         tmp_id = int(row['id'])
#     except:
#         print(i, row['id'])
        
negative_samples = negative_samples.loc[negative_samples['id']!= '1020160014413 (1)']

# for i, row in negative_samples.iterrows():
#     try:
#         tmp_id = int(row['id'])
#     except:
#         print(i, row['id'])
        
negative_samples = negative_samples.loc[~negative_samples['id'].isin(sample_data['id'])]
sample_data = pd.concat([sample_data, negative_samples], axis=0)

# add labels column
sample_data['labels'] = ""
sample_data.loc[sample_data['id']=='1020180014052', 'labels'] = 'source'
sample_data.loc[sample_data['id'].isin(['1020050097605','1020177009557', '1020120156759']), 'labels'] = 'target'
sample_data['labels'] = sample_data['labels'].fillna("negative")
sample_data.to_excel("data_preprocess/sample_data.xlsx")

# Baseline: text only embedding search

- Result: using 3 golden targets and 96 negative samples and 1 source, within top 4 retrieved chunks, all of them are from golden related patents.
- source: 1020180014052 (10-1990984 patent)
- target: 
    - KR1020070041937 A* (1020050097605 app)
    - KR1020170071490 A* (10-2017-7009557 app)
    - KR101414532 B1 (10-2012-0156759 app)
- negative:
    - all others from 100 crawled patents

In [ ]:


def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string("tiktoken is great!", "cl100k_base")

In [ ]:
# TokenTextSplitter -> split by number of tokens. use tokenizer from OpenAI
# https://platform.openai.com/docs/guides/embeddings/what-are-embeddings
# cl100k_base is the tokenizer for the latest embedding (v2)
text_splitter = TokenTextSplitter.from_tiktoken_encoder("cl100k_base",
                                                        chunk_size=500, 
                                                        chunk_overlap=10)

In [ ]:
patent_chunk_dicts=[]
text_columns = ['요약', '청구범위', '기술분야', '배경기술', '해결하려는과제', '과제의해결수단',
       '발명의효과', '도면의간단한설명', '발명을실시하기위한구체적인내용', '부호의설명']

for i, row in sample_data.iterrows():
    patent_dict = dict( 
        # 특허 번호 따기
        application_number = str(row.id), # 출원 번호
        publication_number = '', # 공개 번호
        patent_number = '', # 등록 번호
        chunks = [],
    )
    print(f"patent {row.id} 's chunk sizes")
    
    # text column들을 돌면서, chunking 하기
    chunks = []
    drawing_nums_list = []
    refs_list = []
    for col in text_columns:
        if (col in ['도면의간단한설명', '부호의설명']) or (str(row[col]) == 'nan'): # 도면의 간단한 설명, 부호의 설명은 embed 하지 않음!
            continue
        curr_section_chunks = text_splitter.create_documents([str(row[col])], [{"application_number": str(row.id)}])
        chunks.extend(curr_section_chunks)
        
        print(col, len(curr_section_chunks), end=' | ')
        
        # # chunk 별로 reference 발생한 부호 찾기
        # for chnk in curr_section_chunks:
        #     refs = extract_ref_from_text(chnk)
        #     refs_list.append(refs)
            
        #     #  부호가 있었으면 drawing number 로 한번 또 치환하기
        #     if len(refs) > 0: # if not empty, convert found references to drawing numbers
        #         drawing_nums = convert_refs_to_drawing_num(refs, num2drawing_dicts[str(row.id)]['num2drawing'])
        #     else:
        #         drawing_nums = []
        #     drawing_nums_list.append(drawing_nums)
                
        
    # patent_dict['chunks'] = list(zip(chunks, zip(refs_list, drawing_nums_list)))
    patent_dict['chunks'] = chunks
    print()
    patent_chunk_dicts.append(patent_dict)    
    

In [ ]:
json.dump(patent_chunk_dicts, open('data_preprocess/sample_chunk_data.json', 'w'), ensure_ascii = False )

In [ ]:
all_chunks = [x['chunks'] for x in patent_chunk_dicts if x['application_number'] != '1020180014052']
all_chunks = sum(all_chunks, [])
# print(len(all_chunks))
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents([all_chunks[0]], embedding=embeddings)
for chunk in tqdm(all_chunks[1:]):  # used for loop since if input all chunks at once, RATE LIMIT error occurred.
    vectorstore.add_documents([chunk])
    # time.sleep(2)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 7166/7166 [42:46<00:00,  2.79it/s]  


In [ ]:
len(all_chunks)

192

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
retrieved_docs = retriever.invoke(
    "냉각  제어  모듈,  이를  포함하는  손목  냉각  밴드  및  웨어러블  냉각  장치가  개시된다.  \
    본  발명의  일  측면에따르면, 신체에 냉감을 일으키는 냉각 제어 모듈로서, 흡열표면과 발열표면을 가지는 열전소자와; \
    상기 열전소자에 전원을 공급하기 위한 전원 공급부와; 상기 열전소자의 발열표면에 부착되어 열을 발산하는 방열부재(放熱部材)를  포함하며,\
    상기  방열부재는,  일정  두께를  갖고  두께  방향으로  상기  열을  이동시키는  열분해  흑연  패드(pyrolytic graphite pad)와 \
    금속 방열판이 교대로 적층되어 형성되는, 냉각 제어 모듈이 제공된다."
)
print(retrieved_docs[0].page_content)

한 플렉서블 열전 모듈을 구비한 의류에서 플렉서블 열전 모듈 및 제어부가 사시도로 도시되어 있으며, 도 5에는 본 발명에 의한 플렉서블 열전 모듈을 구비한 의류에서 플렉서블 열전 모듈이 측면도로 도시되어 있다.이들 도면에 의하면, 본 발명의 플렉서블 열전 모듈을 구비한 의류는 의류(100), 플렉서블 열전 모듈(200) 및제어부(300)를 포함한다.의류(100)는 사용자가 착용할 수 있도록 하는 상의 또는 하의를 포함하며, 후술할 플렉서블 열전 모듈(200)이선택적인 위치에 탈부착 될 수 있도록 내피에 수용부가 구비된다. 이때, 상기 의류(100)는 상의인 것으로 예시하며, 하의나 신체의 일부에 착용할 수 있는 형태로도 대체할 수 있다.플렉서블  열전  모듈(200)은  의류(100)의  내피에  장착되어  전원이 인가되면 펠티어(Peltier)  효과에 따라 일측접합부에서 열을 흡수하고 타측 접합부에서 열을 발산한다. 여기서, 열전 모듈은 열에너지와 전기에너지의 상호변환이 가능한 친환경적인 에너지재료로써, 알루미나 등의 세라


In [ ]:
retrieved_docs

[Document(page_content='한 플렉서블 열전 모듈을 구비한 의류에서 플렉서블 열전 모듈 및 제어부가 사시도로 도시되어 있으며, 도 5에는 본 발명에 의한 플렉서블 열전 모듈을 구비한 의류에서 플렉서블 열전 모듈이 측면도로 도시되어 있다.이들 도면에 의하면, 본 발명의 플렉서블 열전 모듈을 구비한 의류는 의류(100), 플렉서블 열전 모듈(200) 및제어부(300)를 포함한다.의류(100)는 사용자가 착용할 수 있도록 하는 상의 또는 하의를 포함하며, 후술할 플렉서블 열전 모듈(200)이선택적인 위치에 탈부착 될 수 있도록 내피에 수용부가 구비된다. 이때, 상기 의류(100)는 상의인 것으로 예시하며, 하의나 신체의 일부에 착용할 수 있는 형태로도 대체할 수 있다.플렉서블  열전  모듈(200)은  의류(100)의  내피에  장착되어  전원이 인가되면 펠티어(Peltier)  효과에 따라 일측접합부에서 열을 흡수하고 타측 접합부에서 열을 발산한다. 여기서, 열전 모듈은 열에너지와 전기에너지의 상호변환이 가능한 친환경적인 에너지재료로써, 알루미나 등의 세라', metadata={'application_number': '1020120156759'}),
 Document(page_content='본 발명은 열전소자를 이용한 백라이트 유닛의 냉각 시스템 및 냉각 방법에 관한 것으로서, 특히 백라이트유닛으로부터 전달된 열을 열전소자에 의해 냉각시키고 발열함으로써 방열팬, 방열핀등의 추가적인 구성이없이 간단히 백라이트 유닛을 냉각할 수 있다는 이점이 있다.이를 위하여 본 발명은 백라이트 유닛과 연결되어 상기 백라이트 유닛에서 발생하는 열을 냉각하는 열전소자를 이용하는 백라이트 유닛의 냉각 시스템에 있어서, 상기 백라이트 유닛의 열원으로부터 열을 흡수하여열전소자에 열을 전달하는 내부 열흡수부; 그 일면은 상기 내부 열흡수부로부터 열을 흡수하여 냉각하고,다른 면은 흡수한 열을 방출하는 열전소자; 및 상기 열전소자로부터 방출된 열을 흡수하여 외부로 방출하는 외부

In [ ]:
# 또다른 방법 
query = "냉각  제어  모듈,  이를  포함하는  손목  냉각  밴드  및  웨어러블  냉각  장치가  개시된다.  \
    본  발명의  일  측면에따르면, 신체에 냉감을 일으키는 냉각 제어 모듈로서, 흡열표면과 발열표면을 가지는 열전소자와; \
    상기 열전소자에 전원을 공급하기 위한 전원 공급부와; 상기 열전소자의 발열표면에 부착되어 열을 발산하는 방열부재(放熱部材)를  포함하며,\
    상기  방열부재는,  일정  두께를  갖고  두께  방향으로  상기  열을  이동시키는  열분해  흑연  패드(pyrolytic graphite pad)와 \
    금속 방열판이 교대로 적층되어 형성되는, 냉각 제어 모듈이 제공된다."
docs = vectorstore.similarity_search(query)

In [ ]:
docs

[Document(page_content='한 플렉서블 열전 모듈을 구비한 의류에서 플렉서블 열전 모듈 및 제어부가 사시도로 도시되어 있으며, 도 5에는 본 발명에 의한 플렉서블 열전 모듈을 구비한 의류에서 플렉서블 열전 모듈이 측면도로 도시되어 있다.이들 도면에 의하면, 본 발명의 플렉서블 열전 모듈을 구비한 의류는 의류(100), 플렉서블 열전 모듈(200) 및제어부(300)를 포함한다.의류(100)는 사용자가 착용할 수 있도록 하는 상의 또는 하의를 포함하며, 후술할 플렉서블 열전 모듈(200)이선택적인 위치에 탈부착 될 수 있도록 내피에 수용부가 구비된다. 이때, 상기 의류(100)는 상의인 것으로 예시하며, 하의나 신체의 일부에 착용할 수 있는 형태로도 대체할 수 있다.플렉서블  열전  모듈(200)은  의류(100)의  내피에  장착되어  전원이 인가되면 펠티어(Peltier)  효과에 따라 일측접합부에서 열을 흡수하고 타측 접합부에서 열을 발산한다. 여기서, 열전 모듈은 열에너지와 전기에너지의 상호변환이 가능한 친환경적인 에너지재료로써, 알루미나 등의 세라', metadata={'application_number': '1020120156759'}),
 Document(page_content='본 발명은 열전소자를 이용한 백라이트 유닛의 냉각 시스템 및 냉각 방법에 관한 것으로서, 특히 백라이트유닛으로부터 전달된 열을 열전소자에 의해 냉각시키고 발열함으로써 방열팬, 방열핀등의 추가적인 구성이없이 간단히 백라이트 유닛을 냉각할 수 있다는 이점이 있다.이를 위하여 본 발명은 백라이트 유닛과 연결되어 상기 백라이트 유닛에서 발생하는 열을 냉각하는 열전소자를 이용하는 백라이트 유닛의 냉각 시스템에 있어서, 상기 백라이트 유닛의 열원으로부터 열을 흡수하여열전소자에 열을 전달하는 내부 열흡수부; 그 일면은 상기 내부 열흡수부로부터 열을 흡수하여 냉각하고,다른 면은 흡수한 열을 방출하는 열전소자; 및 상기 열전소자로부터 방출된 열을 흡수하여 외부로 방출하는 외부

# Preprocessing

**logs**

23.11.15

- 1) problem with extracting reference (number) from texts -> some reference codes are not just numbers. it's number + alphabet e.g. 202a, 202b, or sometimes just uppercase alphabets e.g. T, SR~SZ
- 2) so I tried to first extract the reference codes from "부호에 대한 설명" column. (Because then I can find those specific reference codes inside of texts.) But extracting reference codes from "부호에 대한 설명" was difficult because currently all \<code\>: \<description\> string pairs are concatenated without \n and the parsing reference codes by just relying on regular expression was almost impossible. (some reference codes were uppercase alphabets, but including uppercase alphabets for codes caused problems.) So I asked Mooho if he could leave \n characters left for "부호에 대한 설명" section. He said yes, and I paused developing regex rules further.  

23.11.16
- I proceeded with some noise in reference extraction. 
- I also added some negative samples (3 random samples that are not prior arts of the source patent.)

In [116]:
sample_data = pd.read_excel('data_preprocess/sample_data.xlsx',index_col=0)

## image to numbers, numbers to image
num2drawing_dicts = get_number2drawing_dict('/Users/hayley/Documents/p4ds/patent_search/data_preprocess/mock_data.json')

In [ ]:
# c.f. hupd에서 쓴 컬럼들 
#     "abstract": "...", # 요약 -> 우리나라 특허에는 abstract가 따로 있는거 같진 않고 요약 1개 섹션임
#     "claims": "...", # 청구범위
#     "background": "...", # 기술분야 + 배경기술
#     "summary": "...", # 요약? 
#     "full_description": "..." # 해결하려는과제 + 과제의해결수단 + 발명의효과 + 발명을실시하기위한구체적인내용???

In [71]:
sample_data.columns 
# text embedding 대상이 되는 컬럼
# '요약', '청구범위', '기술분야', '배경기술', '해결하려는과제', '과제의해결수단',
#        '발명의효과', '도면의간단한설명', '발명을실시하기위한구체적인내용', '부호의설명'

# 각 섹션을 따로 따로 임베딩하는게 나을지 아니면은, 몇 섹션은 합치는게 나을지 고민이다. -> 일단 빠르게 ㄱㄱ!

Index(['id', '요약', '대표도', '청구범위', '기술분야', '배경기술', '해결하려는과제', '과제의해결수단',
       '발명의효과', '도면의간단한설명', '발명을실시하기위한구체적인내용', '부호의설명'],
      dtype='object')

In [143]:
text_columns = ['요약', '청구범위', '기술분야', '배경기술', '해결하려는과제', '과제의해결수단',
       '발명의효과', '도면의간단한설명', '발명을실시하기위한구체적인내용', '부호의설명']

In [80]:
# # c.f. 컬럼별 길이 분포 
# all_data = pd.read_excel('/Users/hayley/Documents/p4ds/patent_search/pdf_process/data_large.xlsx', index_col=0)
# for col in all_data.columns:
#     try:
#         if "extracted_numbers" not in col:
#             lens = all_data[col].apply(lambda x: len(str(x)))
#             print(col, lens.mean().round(2))
#     except:
#         continue
# # id 13.04
# # 요약 301.69
# # 대표도 395.03
# # 청구범위 3565.65
# # 기술분야 150.39
# # 배경기술 1713.51
# # 해결하려는과제 210.42
# # 과제의해결수단 1653.4
# # 발명의효과 209.81
# # 도면의간단한설명 1206.95
# # 발명을실시하기위한구체적인내용 17958.91
# # 부호의설명 188.81

In [221]:
patent_chunk_dicts=[]

for i, row in sample_data.iterrows():
    patent_dict = dict( 
        # 특허 번호 따기
        application_number = str(row.id), # 출원 번호
        publication_number = '', # 공개 번호
        patent_number = '', # 등록 번호
        chunks = [],
        # chunks_wo_drawing_and_numbers_desc = []
    )
    print(f"patent {row.id} 's chunk sizes")
    
    # text column들을 돌면서, chunking 하기
    chunks = []
    drawing_nums_list = []
    refs_list = []
    # chunks_wo_drawing_desc = [] # 도면의 간단한 설명, 부호의 설명 제외.. false positive 가 생기지는 않을까?
    for col in text_columns:
        if (col in ['도면의간단한설명', '부호의설명']) or (str(row[col]) == 'nan'):
            continue
        curr_section_chunks = text_splitter.create_documents([str(row[col])], [{"application_number": str(row.id)}])
        chunks.extend(curr_section_chunks)
        
        # chunks_wo_drawing_desc.extend(curr_section_chunks)
        print(col, len(curr_section_chunks), end=' | ')
        
        # chunk 별로 reference 발생한 부호 찾기
        for chnk in curr_section_chunks:
            refs = extract_ref_from_text(chnk)
            refs_list.append(refs)
            
            #  부호가 있었으면 drawing number 로 한번 또 치환하기
            if len(refs) > 0: # if not empty, convert found references to drawing numbers
                drawing_nums = convert_refs_to_drawing_num(refs, num2drawing_dicts[str(row.id)]['num2drawing'])
            else:
                drawing_nums = []
            drawing_nums_list.append(drawing_nums)
                
        
    # patent_dict['chunks'] = list(zip(chunks, zip(refs_list, drawing_nums_list)))
    patent_dict['chunks'] = chunks
    # patent_dict['chunks_wo_drawing_and_numbers_desc'] = chunks_wo_drawing_desc
    print()
    patent_chunk_dicts.append(patent_dict)    
    

patent 1020157015204 's chunk sizes
요약 1 | 청구범위 6 | 기술분야 1 | 배경기술 2 | 발명을실시하기위한구체적인내용 70 | 
patent 1020120156759 's chunk sizes
요약 1 | 기술분야 1 | 배경기술 3 | 해결하려는과제 1 | 과제의해결수단 1 | 발명의효과 1 | 발명을실시하기위한구체적인내용 14 | 
patent 1020200149050 's chunk sizes
요약 1 | 청구범위 3 | 기술분야 1 | 배경기술 6 | 해결하려는과제 1 | 과제의해결수단 1 | 발명의효과 1 | 발명을실시하기위한구체적인내용 8 | 
patent 1020180014052 's chunk sizes
요약 1 | 청구범위 5 | 기술분야 1 | 배경기술 2 | 해결하려는과제 1 | 과제의해결수단 4 | 발명의효과 1 | 발명을실시하기위한구체적인내용 19 | 
patent 1020177009557 's chunk sizes
요약 1 | 청구범위 3 | 기술분야 1 | 배경기술 1 | 해결하려는과제 1 | 과제의해결수단 8 | 발명의효과 1 | 발명을실시하기위한구체적인내용 14 | 
patent 1020127022798 's chunk sizes
요약 1 | 청구범위 12 | 기술분야 1 | 배경기술 27 | 발명을실시하기위한구체적인내용 46 | 
patent 1020050097605 's chunk sizes
요약 1 | 청구범위 3 | 배경기술 3 | 해결하려는과제 1 | 발명의효과 1 | 발명을실시하기위한구체적인내용 8 | 
patent 1020080110005 's chunk sizes
요약 1 | 기술분야 1 | 배경기술 24 | 
patent 1020107017567 's chunk sizes
요약 1 | 청구범위 8 | 기술분야 1 | 배경기술 3 | 해결하려는과제 1 | 과제의해결수단 1 | 발명의효과 1 | 발명을실시하기위한구체적인내용 15 | 
patent 1020110134350 's chu

In [191]:
json.dump(patent_chunk_dicts, open('data_preprocess/sample_chunk_data.json', 'w'), ensure_ascii = False )

In [181]:
# for col in select_columns:
#     sample_data[f"{col}_ref"] = sample_data[col].apply(extract_ref_from_text)

# sample_data['도면의간단한설명_dict'] = sample_data['도면의간단한설명'].apply(extract_description_for_image)

# sample_data.loc[:,'부호의설명_dict'] = sample_data['부호의설명'].apply(extract_description_for_code)

# for col in select_columns:
#     sample_data[f"{col}_ref_drawing"] = sample_data.apply(convert_ref_to_drawing_num, args=(col,image_df), axis=1)

# OCR

- Tesseract : 잘 못함
- 다른 오픈소스 : pyocr -> tesseract랑 똑같음 / calamari-ocr -> tensorflow 설치해야 되고 등등 maintain이 잘 안되는 패키지 인듯
- GCP vision api : 일단 ui 에서 테스트 해보고 copy json output 버튼 눌러가지고 json output 복사해와서 json output에서 텍스트만 파싱하는 것만 짬

In [ ]:
import pytesseract
from PIL import Image

## tesseract를 먼저 다운받고, 다운받은 경로를 넣어주어야 함.
## You need to first download tesseract & insert the path to the exe file below.
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Cellar/tesseract/5.3.3/bin/tesseract'

In [47]:
# text image to string
from glob import glob

image_paths = glob('data_preprocess/image/*/*.png')

for img in image_paths:
    print(img)
    print(pytesseract.image_to_string(Image.open(img), lang='eng'))

data_preprocess/image/pdf_1020120156759/p8-6.png

data_preprocess/image/pdf_1020120156759/p7-31.png
A256
ART
129
A120
A158
123,
121

133

A35
437

10 130110 145: 147
140- 143


data_preprocess/image/pdf_1020120156759/p7-30.png
110 5 110

1 ~_-f


data_preprocess/image/pdf_1020120156759/p0-6.png

data_preprocess/image/pdf_1020120156759/p9-39.png
200

212
214a

222 224a 226 228

data_preprocess/image/pdf_1020120156759/p8-35.png
200

300

320
310


data_preprocess/image/pdf_1020180014052/p14-56.png

data_preprocess/image/pdf_1020180014052/p12-6.png

data_preprocess/image/pdf_1020180014052/p15-60.png

data_preprocess/image/pdf_1020180014052/p0-6.png

data_preprocess/image/pdf_1020180014052/p13-52.png

data_preprocess/image/pdf_1020180014052/p11-43.png

data_preprocess/image/pdf_1020180014052/p11-44.png

data_preprocess/image/pdf_1020180014052/p11-45.png

data_preprocess/image/pdf_1020177009557/p13-66.png
1

4

5

\


data_preprocess/image/pdf_1020177009557/p13-67.png
La

4 a, 3 i
cs ot “
Q

In [71]:
for i in json_output['fullTextAnnotation']['pages'][0]['blocks']:
    for j in i['paragraphs']:
        for k in j['words']:
            curr_word = ''
            for l in k['symbols']:
                curr_word += l['text']
            if curr_word.startswith('-'):
                curr_word = curr_word[1:]
            print(curr_word)

30
100
40
20
10


In [76]:
annotations = json_output['textAnnotations'][0]['description'].split('\n')
print(annotations)

['-30', '100', '40', '20', '-10']


In [78]:
annotations = json_output['textAnnotations'][0]['description'].split('\n')
print(annotations)

['15', '***************', '17']
